<a href="https://colab.research.google.com/github/AlirezaAhadipour/NN-from-scratch/blob/main/makemore_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
words = open('names.txt', 'r').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [ ]:
# build the vocabulary of chars and mapping to/from integers
chars = sorted(list(set(''.join(words))))

str_to_int = {s:i+1 for i , s in enumerate(chars)}
str_to_int['.'] = 0

int_to_str = {i:s for s, i in str_to_int.items()}

print(int_to_str)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [18]:
# build the dataset
block_size =3   # num of chars we take to predict the next one
X, y = [], []

for word in words[:3]:
  print(word)

  context = [0] * block_size
  for char in word + '.':
    ix = str_to_int[char]
    X.append(context)
    y.append(ix)

    print(''.join(int_to_str[i] for i in context), '--->', int_to_str[ix])
    context = context[1:] + [ix]

X = torch.tensor(X)
y = torch.tensor(y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .


In [19]:
X.shape, X.dtype, y.shape, y.dtype

(torch.Size([16, 3]), torch.int64, torch.Size([16]), torch.int64)

In [50]:
C = torch.randn((27,2))

embd = C[X]
embd.shape

torch.Size([16, 3, 2])

In [51]:
embd.view(16,6)

tensor([[ 0.1563,  0.2031,  0.1563,  0.2031,  0.1563,  0.2031],
        [ 0.1563,  0.2031,  0.1563,  0.2031,  0.8598,  1.6875],
        [ 0.1563,  0.2031,  0.8598,  1.6875,  0.8771, -0.0774],
        [ 0.8598,  1.6875,  0.8771, -0.0774,  0.8771, -0.0774],
        [ 0.8771, -0.0774,  0.8771, -0.0774,  1.4245, -0.1805],
        [ 0.1563,  0.2031,  0.1563,  0.2031,  0.1563,  0.2031],
        [ 0.1563,  0.2031,  0.1563,  0.2031,  0.5053, -1.4254],
        [ 0.1563,  0.2031,  0.5053, -1.4254,  0.3965,  0.4448],
        [ 0.5053, -1.4254,  0.3965,  0.4448, -0.4226,  1.5325],
        [ 0.3965,  0.4448, -0.4226,  1.5325,  0.7715, -1.3335],
        [-0.4226,  1.5325,  0.7715, -1.3335, -0.4226,  1.5325],
        [ 0.7715, -1.3335, -0.4226,  1.5325,  1.4245, -0.1805],
        [ 0.1563,  0.2031,  0.1563,  0.2031,  0.1563,  0.2031],
        [ 0.1563,  0.2031,  0.1563,  0.2031,  1.4245, -0.1805],
        [ 0.1563,  0.2031,  1.4245, -0.1805,  0.7715, -1.3335],
        [ 1.4245, -0.1805,  0.7715, -1.3

In [52]:
W1 = torch.randn(6, 100)
b1 = torch.randn(100)

h = torch.tanh(embd.view(embd.shape[0], 6) @ W1 + b1)
h

tensor([[-0.0963, -0.4472, -0.8849,  ..., -0.6977,  0.0796, -0.8813],
        [ 0.6819,  0.3269, -0.9958,  ..., -0.0842, -0.9987, -0.9993],
        [ 0.4496, -0.5242, -0.9963,  ..., -0.5688, -0.9237,  0.7085],
        ...,
        [ 0.8764,  0.4791, -0.8405,  ..., -0.4494, -0.9456, -0.5065],
        [-0.3190,  0.8170, -0.4501,  ..., -0.7751,  0.7802, -0.4318],
        [ 0.7092,  0.9332,  0.7269,  ..., -0.9526, -0.9901, -0.9532]])

In [53]:
W2 = torch.randn(100, 27)
b2 = torch.randn(27)

logits = h @ W2 + b2
logits

tensor([[-9.9267e+00, -2.6273e+00,  4.9140e+00, -4.2383e+00, -2.0227e+00,
         -2.5486e+00, -1.0165e+01,  9.2521e+00,  4.5801e+00, -3.9381e+00,
         -9.1730e-01, -7.2660e+00, -1.0691e+01, -7.8783e+00, -6.3563e+00,
         -9.7888e-01, -2.0791e+00,  1.8570e+00,  2.6425e+00, -2.2188e+00,
         -8.8841e+00,  1.5901e+01, -1.1087e+01,  9.8384e+00,  2.7424e+00,
          4.7518e+00,  7.5922e-01],
        [-8.8270e+00, -2.3601e+00,  4.2100e+00, -5.4560e+00,  1.9522e+00,
         -6.8630e+00, -9.2107e+00,  1.9005e+01,  1.5722e+01, -5.4658e+00,
         -3.1908e-01, -5.1307e+00, -1.5326e+01,  4.0116e+00,  7.7939e+00,
          7.8528e-01,  4.6285e+00, -1.6431e+00,  2.4217e+01, -7.7707e+00,
         -6.4581e+00,  1.0153e+01, -3.0972e+00,  7.0060e+00,  3.4978e+00,
          8.5612e-01,  7.8188e+00],
        [-1.4341e+00, -7.7178e+00,  1.0679e+01,  3.2498e+00,  1.8306e-01,
         -1.7813e+01, -1.8201e+00,  8.1347e+00, -4.6498e+00, -8.0288e+00,
          2.3206e+00, -4.2729e+00, -1.50

In [54]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)
loss = -prob[torch.arange(embd.shape[0]), y].log().mean()
loss

tensor(21.6769)

In [55]:
F.cross_entropy(logits, y)

tensor(21.6769)

In [97]:
# build the dataset
block_size =3   # num of chars we take to predict the next one
X, y = [], []

for word in words:

  context = [0] * block_size
  for char in word + '.':
    ix = str_to_int[char]
    X.append(context)
    y.append(ix)

    context = context[1:] + [ix]

X = torch.tensor(X)
y = torch.tensor(y)

In [98]:
# parameters
C = torch.randn((27,2))
W1 = torch.randn(6, 100)
b1 = torch.randn(100)
W2 = torch.randn(100, 27)
b2 = torch.randn(27)

parameters = [C, W1, b1, W2, b2]

n_param = sum(p.nelement() for p in parameters)
print(n_param)

3481


In [99]:
for param in parameters:
  param.requires_grad = True

for _ in range(1000):
    # minibatch
    ix = torch.randint(0, X.shape[0], (32,))   #batch of size 32

    # forward pass
    embd = C[X[ix]]
    h = torch.tanh(embd.view(embd.shape[0], 6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, y[ix])

    # backward pass
    for param in parameters:
      param.grad = None

    loss.backward()

    # update
    for param in parameters:
      param.data += -0.1 * param.grad

# print(loss.item())

In [100]:
embd = C[X]
h = torch.tanh(embd.view(embd.shape[0], 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, y)
loss.item()

2.592813491821289